In [2]:
import pandas as pd
import nltk
import numpy
import re
from nltk.stem import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import pos_tag
from sklearn.metrics import pairwise_distances
from nltk import word_tokenize
from nltk.corpus import stopwords

In [3]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
df=pd.read_csv('./dialogs.txt',sep="	")
df.head()

,Context,Text Response
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.no problem....,NaN
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.


In [5]:
df.shape[0]

3725

In [6]:
df.ffill(axis=0, inplace=True)
df.head(5)

,Context,Text Response
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.no problem....,i'm pretty good. thanks for asking.
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.


In [7]:
df1=df.head(10)
def step1(x):
    for i in x:
        a=str(i).lower
        p=re.sub(r'[^a-z0-9]',' ',a)
        print(p)

Tokenizing

In [8]:
s='tell me about yourself'
words=word_tokenize(s)
print(words)

['tell', 'me', 'about', 'yourself']


In [9]:
pos_tag(nltk.word_tokenize(s), tagset = None)

[('tell', 'VB'), ('me', 'PRP'), ('about', 'IN'), ('yourself', 'PRP')]

In [10]:
lemma = wordnet.WordNetLemmatizer()
lemma.lemmatize('absorbed', pos = 'v')

'absorb'

In [11]:
def text_normalization(text):
    text = str(text).lower()
    spl_char_text = re.sub(r'[^ a-z]','', text)
    tokens = nltk.word_tokenize(spl_char_text)
    lema = wordnet.WordNetLemmatizer()
    tags_list = pos_tag(tokens, tagset=None)
    lema_words=[]
    for token, pos_token in tags_list:
        if pos_token.startswith('V'):
            pos_val='v'
        elif pos_token.startswith('J'):
            pos_val='a'
        elif pos_token.startswith('R'):
            pos_val='r'
        else:
            pos_val='n'
        lema_token=lema.lemmatize(token, pos_val)
        lema_words.append(lema_token)

    return " ".join(lema_words)

In [12]:
text_normalization("telling you some stuff about myself")

'tell you some stuff about myself'

In [13]:
df['lemmatized_text'] = df['Context'].apply(text_normalization)
df.head()

,Context,Text Response,lemmatized_text
0,"hi, how are you doing?",i'm fine. how about yourself?,hi how be you do
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.,im fine how about yourself
2,i'm pretty good. thanks for asking.no problem....,i'm pretty good. thanks for asking.,im pretty good thanks for askingno problem so ...
3,no problem. so how have you been?,i've been great. what about you?,no problem so how have you be
4,i've been great. what about you?,i've been good. i'm in school right now.,ive be great what about you


BOW

In [14]:
cv = CountVectorizer()
X = cv.fit_transform(df['lemmatized_text']).toarray()

In [17]:
features = cv.get_feature_names_out()
df_bow = pd.DataFrame(X, columns = features)
df_bow.head()

,able,about,absolutely,accident,ache,across,act,action,activity,actor,...,youll,young,your,youre,yours,yourself,youve,yuck,zip,zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [19]:
Question = 'Will you help me and tell me about yourself more'

In [20]:
Q = []
a = Question.split()
for i in a:
    if i in stop:
        continue
    else:
        Q.append(i)
    b=" ".join(Q)

In [21]:
Question_lemma = text_normalization(b)
Question_bow = cv.transform([Question_lemma]).toarray()

TFIDF

In [22]:
tfidf = TfidfVectorizer()
x_tfidf = tfidf.fit_transform(df['lemmatized_text']).toarray()

In [23]:
df_tfidf = pd.DataFrame(x_tfidf, columns=tfidf.get_feature_names_out())
df_tfidf.head()

,able,about,absolutely,accident,ache,across,act,action,activity,actor,...,youll,young,your,youre,yours,yourself,youve,yuck,zip,zoo
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.331282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.572114,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.424174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [24]:
Question_tfidf = tfidf.transform([Question_lemma]).toarray()

In [25]:
cos = 1 - pairwise_distances(df_tfidf, Question_tfidf, metric='cosine')
cos

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [26]:
index_value1 = cos.argmax()
index_value1

1875

In [27]:
Question

'Will you help me and tell me about yourself more'

In [28]:
df['Text Response'].loc[index_value1]

"okay, i'll get a bucket."

In [29]:
def chat_tfidf(text):
    lemma = text_normalization(text)
    tf = tfidf.transform([lemma]).toarray()
    cos = 1 - pairwise_distances(df_tfidf, tf, metric='cosine')
    index_value = cos.argmax()
    return df['Text Response'].loc[index_value]

In [30]:
chat_tfidf('hi!')

"i'm fine. how about yourself?"

In [31]:
chat_tfidf('What is the meaning of life?')

'it sure is.'

In [32]:
chat_tfidf('Where is it?')

'i have to walk the dog.'